In [ ]:
# happy-agents Chapter 3 exercise

In [ ]:
# uninstall previous torch versions
# !pip uninstall torch torchvision torchaudio -y

In [7]:
import torch
print(torch.__version__)

2.10.0+cu126


In [ ]:
# install Hugging Face transformers and PyTorch with CUDA support
# !pip install transformers 
# !pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu126

In [8]:
# load model and tokenizer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "Qwen/Qwen3-0.6B"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
print("Model and tokenizer loaded successfully.")

Using device: cuda


Loading weights: 100%|██████████| 311/311 [00:00<00:00, 974.05it/s, Materializing param=model.norm.weight]                               
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Model and tokenizer loaded successfully.


In [9]:
# prepare dialogue input
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "你好，介绍一下你自己。"},
]

# format input for causal LM
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# encode input text
model_inputs = tokenizer([text], return_tensors="pt").to(device)

print("编码后的输入示例：")
print(model_inputs)



编码后的输入示例：
{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 108386,   3837, 109432, 107828,
           1773, 151645,    198, 151644,  77091,    198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


In [10]:
# generate response
# max_new_tokens controls the length of the generated response
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)

# cut off the input part to get only the generated response
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# decode generated tokens to text
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("\n生成的回复：")
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



生成的回复：
<think>
好的，用户让我介绍一下自己。首先，我需要确认自己是什么。作为AI助手，我主要提供帮助和支持，但具体信息可能需要用户进一步说明。我应该保持友好和开放的态度，同时确保信息准确。用户可能只是想了解我的功能或背景，所以需要简明扼要地回答，避免冗长。另外，要确保回答符合用户的需求，可能还需要询问是否有其他问题需要帮助。最后，保持语气亲切自然，让用户感到被重视。
</think>

您好！我是您的AI助手，随时为您提供帮助和支持。如果您有任何问题或需要帮助，请随时告诉我！有什么我可以帮您解决的吗？😊
